In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [42]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls), len(files)

100%|█████████████████████████████████| 45782/45782 [00:00<00:00, 110407.90it/s]


(441780, 45782)

In [31]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [35]:
urls[0]

'https://www.orientaldaily.com.my/news/Travel/2021/07/05/422417'

In [34]:
soup.find('h1').text

'PETRA Riyaz经营标志性度假村\u3000打造生活方式酒店品牌'

In [37]:
soup.find('div', {'itemprop':"articleBody"}).text

"\nPETRA Riyaz现今将经营一系列以生活方式为导向的酒店品牌，每个品牌都是专为小众客户群，无论是现代、高级或豪华的旅行者而打造。\n该公司旗下品牌包括Riyaz、dash、The Pure、Altara和 Tan'Yaa。继PETRA集团完成Riyaz酒店及度假村的51.5%股份合并后，这整个酒店和度假村市值接近3亿令吉。\nPETRA Riyaz将经营标志性度假村，包括五星级豪华酒店The Riyaz Lavanya Resort & Villas和浮罗交怡的dash度假村；布城的dash Box和的Tan'Yaa酒店；印尼巴厘岛的dash Hotel Seminyak酒店；以及位于越南岘港的Altara Suites by Riyaz。\n\n\nPETRA Riyaz首席执行员拿督斯里沙欣表示，尽管疫情造成许多不确定性，但PETRA Riyaz已经在致力于扩展其各种酒店品牌。\n\n\nAdvertisement\n\n\n位于大马、越南和马尔代夫的新度假村，目前已处于规划阶段。而The dash Hotel KLCC 计划将于2023年初开业，该集团也将其dash品牌业务，扩展到了槟城，柔州新山和马尔代夫。\n尽管在新冠肺炎疫情和行管令的实施下，许多酒店被迫暂时或永久关闭，PETRA集团主席兼首席执行员拿督维诺德却认为，旅游业的复苏迫在眉睫。\n“一旦疫情得到控制，疫苗接种广泛，人们需要休息，家人会想一起去度假，公司会使用酒店重新联系同行并重振他们的愿景。我们将随时准备通过各种酒店，为每个人提供独特而难忘的体验。”\n“更重要的是，我们希望当地旅游业能再次蓬勃发展，并提供许多依赖该行业的人士工作与收入。即便国家边界仍然可能会长时间地关闭，我们仍会尽一份力确保任何一家酒店的工作人员被不裁员。\xa0而在这方面，政府和政府发挥著关键作用。”\n\n\n 酒店\n\n\n追踪东方日报社交平台\n\n\n\n\n\nTelegram\n\n\n\n\n\n\nXiaohongshu logo - Brandlogos.net\n\n\n\n追看热门新闻资讯，请下载东方日报APP\n\n IOS：https://odn.my/ios\n Google Play：https://odn.my/android\n HUAWEI AppGallery：https:

In [38]:
def crawl(url, filename):
    if os.path.exists(filename):
        return [True]
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)

    soup = BeautifulSoup(r.content, "lxml")
    sections = soup.find_all('section')
    try:
        title = soup.find('h1').text
    except:
        title = None
        
    try:
        body = soup.find('div', {'itemprop':"articleBody"}).text
    except:
        body = None
        
    data = {
        'title': title,
        'body': body,
        'url': url,
    }

    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [39]:
!mkdir page

In [46]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 30

for i in tqdm(range(0, len(urls), max_worker)):
    b = urls[i: i + max_worker]
    urls_ = [(b[k], os.path.join('page', f'{i + k}.json')) for k in range(len(b))]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 23%|███████▊                          | 3400/14726 [1:34:52<4:46:30,  1.52s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 61%|████████████████████▋             | 8953/14726 [4:09:03<2:28:11,  1.54s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

